In [1]:
%matplotlib inline

In [2]:
import datetime
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import os
import pandas as pd

In [3]:
# - See what dir we're in
os.getcwd()

'/opt/skipjack-bigeye-separation/python'

In [4]:
import scipy as sp
from reverse_colormap import reverse_colormap
import cmocean

In [5]:
from mpl_toolkits.basemap import Basemap, addcyclic, shiftgrid

KeyError: 'PROJ_LIB'

In [ ]:
dpath = '/opt/skipjack-bigeye-separation/data/'

# Load monthly 5 by 5 degree WCPFC catch and effort data

Variables loaded are:  
lon, lat, time,
skj_c_una, skj_c_log, skj_c_dfad, skj_c_afad, skj_c_oth,  
bet_c_una, bet_c_log, bet_c_dfad, bet_c_afad, bet_c_oth,  
sets_una, sets_log, sets_dfad, sets_afad, sets_oth

In [ ]:
ncfile = dpath + 'WCPFC/5by5deg/ByMonth/PurseSeine_0/WCPFC_purseseine_bysettype_monthly_5deg.nc'
fh = nc.Dataset(ncfile, mode='r') # fh = file handle

In [ ]:
lon = fh.variables['lon'][:]
lat = fh.variables['lat'][:]
time = fh.variables['time'][:] # monthly data starts jan-1967

In [ ]:
# - Dimensions of following variables are time, y, x
skj_c_una = fh.variables['skj_c_una'][:];
skj_c_log = fh.variables['skj_c_log'][:];
skj_c_dfad = fh.variables['skj_c_dfad'][:];
skj_c_afad = fh.variables['skj_c_afad'][:];
skj_c_oth = fh.variables['skj_c_oth'][:];

bet_c_una = fh.variables['bet_c_una'][:];
bet_c_log = fh.variables['bet_c_log'][:];
bet_c_dfad = fh.variables['bet_c_dfad'][:];
bet_c_afad = fh.variables['bet_c_afad'][:];
bet_c_oth = fh.variables['bet_c_oth'][:];

sets_una = fh.variables['sets_una'][:];
sets_log = fh.variables['sets_log'][:];
sets_dfad = fh.variables['sets_dfad'][:];
sets_afad = fh.variables['sets_afad'][:];
sets_oth = fh.variables['sets_oth'][:];

In [ ]:
fh.close()

In [ ]:
# - Check that everything looks as expected
print(lon)
print(type(lon))
print(lat)
print(time.shape)
print(np.shape(skj_c_una))

# Define beginning and end dates for catch data

Variables created are:  
begdate, enddate, timemocatch

In [ ]:
# - Time goes from Jan 1967 - Dec 2017 --> 612 months
begdate = datetime.date(year = 1967, month = 1, day = 1)
enddate = datetime.date(year = 2017, month = 12, day = 1)
timemocatch = pd.date_range(begdate, enddate, freq='MS')
#np.array(timemocatch)

# Load monthly ENSO time series

Variables loaded/calculated are:  
oni, onien, oniln, onitime

In [ ]:
# - Load ONI 
dfoni = pd.read_csv(dpath+'ENSOindices/oni/oniindex1950_2018.txt',names=['Date','ONI']);
#dfoni.head(2)

# - Month fractions corresp to jan, feb, mar, etc. in the ONI txt file
monthfracs = [0,0.0834,0.1666,0.25,0.3334,0.4166,0.5,0.5834,0.6666,0.75,0.8334,0.9166];

# - Get ONI ts over desired timeframe
onibegmoidx = dfoni.index[dfoni['Date'] == (begdate.year + monthfracs[begdate.month-1])][0]
oniendmoidx = dfoni.index[dfoni['Date'] == (enddate.year + monthfracs[enddate.month-1])][0]
oni = dfoni['ONI'].iloc[onibegmoidx:(oniendmoidx+1)].values
onitime = pd.date_range(begdate, enddate, freq='MS') # should be 612 months long

In [ ]:
# - Create EN/LN month classifications
onien = [0]*len(oni) # 1 = el nino month
oniln = [0]*len(oni) # 1 = la nina month

# - Classify El Nino months
cmcounter = 0; # consecutive months counter
for imonth in range(len(oni)):
    if oni[imonth]>=0.5:
        cmcounter=cmcounter+1;
    elif oni[imonth]<0.5:
        cmcounter=0;
    if cmcounter>=5:
        onien[imonth-cmcounter:imonth]=[1]*cmcounter;

# - Classify La Nina months
cmcounter = 0; # consecutive months counter
for imonth in range(len(oni)):
    if oni[imonth]<=-0.5:
        cmcounter=cmcounter+1;
    elif oni[imonth]>-0.5:
        cmcounter=0;
    if cmcounter>=5:
        oniln[imonth-cmcounter:imonth]=[1]*cmcounter;

In [ ]:
type(oni)

In [ ]:
plotensoidx=1

if plotensoidx==1:
    fig = plt.figure(figsize=(11,4))
    plt.plot(onitime,oni)
    plt.plot(onitime,onien)
    plt.plot(onitime,oniln)
    plt.legend(["oni","onien","oniln"])

# Calculate WCPFC total catch, CPUE, effot, bet:skj catch ratio, bet:skj CPUE ratio

Variables calced are:  
skj_c_tot, bet_c_tot, sets_tot, skj_cp_tot, bet_cp_tot, bettoskj_c_tot, bettoskj_cp_tot

In [5]:
skj_c_tot = skj_c_una + skj_c_log + skj_c_dfad + skj_c_afad + skj_c_oth
bet_c_tot = bet_c_una + bet_c_log + bet_c_dfad + bet_c_afad + bet_c_oth
sets_tot = sets_una + sets_log + sets_dfad + sets_afad + sets_oth
skj_cp_tot = skj_c_tot/sets_tot
bet_cp_tot = bet_c_tot/sets_tot
bettoskj_c_tot = bet_c_tot/skj_c_tot
bettoskj_cp_tot = bet_c_tot/skj_cp_tot

NameError: name 'skj_c_una' is not defined

# Calculate mean and ENSO composites of total catch, CPUE, effort bet:skj catch and CPUE

In [6]:
skj_c_tot_mean = np.mean(skj_c_tot,axis=0)
skj_c_tot_en = np.mean(np.squeeze(skj_c_tot[np.where(onien),:,:]),axis=0)
skj_c_tot_ln = np.mean(np.squeeze(skj_c_tot[np.where(oniln),:,:]),axis=0)

bet_c_tot_mean = np.mean(bet_c_tot,axis=0)
bet_c_tot_en = np.mean(np.squeeze(bet_c_tot[np.where(onien),:,:]),axis=0)
bet_c_tot_ln = np.mean(np.squeeze(bet_c_tot[np.where(oniln),:,:]),axis=0)

skj_cp_tot_mean = np.mean(skj_cp_tot,axis=0)
skj_cp_tot_en = np.mean(np.squeeze(skj_cp_tot[np.where(onien),:,:]),axis=0)
skj_cp_tot_ln = np.mean(np.squeeze(skj_cp_tot[np.where(oniln),:,:]),axis=0)

bet_cp_tot_mean = np.mean(bet_cp_tot,axis=0)
bet_cp_tot_en = np.mean(np.squeeze(bet_cp_tot[np.where(onien),:,:]),axis=0)
bet_cp_tot_ln = np.mean(np.squeeze(bet_cp_tot[np.where(oniln),:,:]),axis=0)

sets_tot_mean = np.mean(sets_tot,axis=0)
sets_tot_en = np.mean(np.squeeze(sets_tot[np.where(onien),:,:]),axis=0)
sets_tot_ln = np.mean(np.squeeze(sets_tot[np.where(oniln),:,:]),axis=0)

bettoskj_c_tot_mean = np.mean(bettoskj_c_tot,axis=0)
bettoskj_c_tot_en = np.mean(np.squeeze(bettoskj_c_tot[np.where(onien),:,:]),axis=0)
bettoskj_c_tot_ln = np.mean(np.squeeze(bettoskj_c_tot[np.where(oniln),:,:]),axis=0)

NameError: name 'skj_c_tot' is not defined